In [1]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from catboost import CatBoostClassifier
from xgboost import XGBClassifier

from interpret.glassbox import ExplainableBoostingClassifier
from interpret.glassbox import DecisionListClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import BaggingClassifier

clfs = {
    "CART": DecisionTreeClassifier(random_state=1234, max_depth=1000),
    "RNF": RandomForestClassifier(random_state=1234),
    "XGB": XGBClassifier(use_label_encoder=False),
    "CAT": CatBoostClassifier(random_state=1234),
    "ADA": AdaBoostClassifier(DecisionTreeClassifier(random_state=1234, max_depth=1000)),
    "BAG": BaggingClassifier(DecisionTreeClassifier(random_state=1234, max_depth=1000)),
    "EBM": ExplainableBoostingClassifier(),
    "LR_l2": LogisticRegression(penalty="l2",random_state=1234),
    "GNB": GaussianNB(),
    "LR": LogisticRegression(penalty="none", random_state=1234),
    "DL": DecisionListClassifier(random_state=1234) 
  
}

dataset = 'adult'

import worstcase_helper
import importlib
importlib.reload(worstcase_helper)

preprocess, X, y = worstcase_helper.load_dataset_with_preprocess(dataset)

from sklearn.pipeline import make_pipeline
def make_pipeline_clf(clf_name):
    clf = make_pipeline(
        preprocess,
        clfs[clf_name]
    )
    return clf

clfs_list = []
for clf_id, clf_name in enumerate(clfs):
    clf = make_pipeline_clf(clf_name)
    clf.fit(X, y)
    clfs_list.append(clf)

import pickle

for clf_id, clf_name in enumerate(clfs):
    pickle.dump(clfs_list[clf_id], open(f"adult_{clf_name}.p", 'wb'))


/home/bart/anaconda3/envs/mainenv/lib/python3.9/site-packages/sklearn/pipeline.py:394: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self._final_estimator.fit(Xt, y, **fit_params_last_step)


[17:53:53] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Learning rate set to 0.045587
0:	learn: 0.6453971	total: 385ms	remaining: 6m 24s
1:	learn: 0.6070598	total: 759ms	remaining: 6m 18s
2:	learn: 0.5687926	total: 1s	remaining: 5m 32s
3:	learn: 0.5409262	total: 1.18s	remaining: 4m 52s
4:	learn: 0.5131559	total: 1.31s	remaining: 4m 20s
5:	learn: 0.4893856	total: 1.48s	remaining: 4m 5s
6:	learn: 0.4684633	total: 1.62s	remaining: 3m 50s
7:	learn: 0.4495447	total: 1.85s	remaining: 3m 49s
8:	learn: 0.4352291	total: 2.1s	remaining: 3m 50s
9:	learn: 0.4225243	total: 2.3s	remaining: 3m 47s
10:	learn: 0.4126622	total: 2.6s	remaining: 3m 54s
11:	learn: 0.4023751	total: 2.87s	remaining: 3m 55s
12:	learn: 0.3935590	total: 2.99s	remaining: 3m 46s
13:	learn: 0.3855693	total: 3.12s	remaining: 3m 39s
14:	lea

/home/bart/anaconda3/envs/mainenv/lib/python3.9/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/bart/anaconda3/envs/mainenv/lib/python3.9/site-packages/sklearn/ensemble/_bagging.py:719: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/bart/anaconda3/envs/mainenv/lib/python3.9/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/bart/anaconda3/envs/mainenv/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (

In [2]:
import pickle

for clf_id, clf_name in enumerate(clfs):
    pickle.dump(clfs_list[clf_id], open(f"adult_{clf_name}.p", 'wb'))

In [5]:
# clf_cart_exp = dx.Explainer(clf_cart, X, y, label="CART")
import dalex as dx
clfs_explainers = []
for clf_id, clf_name in enumerate(clfs):
    clf_exp = dx.Explainer(clfs_list[clf_id], X, y, label=clf_name)
    clfs_explainers.append(clf_exp)

Preparation of a new explainer is initiated

  -> data              : 32559 rows 14 cols
  -> target variable   : Parameter 'y' was a pandas.DataFrame. Converted to a numpy.ndarray.
  -> target variable   : 32559 values
  -> model_class       : sklearn.tree._classes.DecisionTreeClassifier (default)
  -> label             : CART
  -> predict function  : <function yhat_proba_default at 0x7fdee00d59d0> will be used (default)
  -> predict function  : Accepts only pandas.DataFrame, numpy.ndarray causes problems.
  -> predicted values  : min = 0.0, mean = 0.241, max = 1.0
  -> model type        : classification will be used (default)
  -> residual function : difference between y and yhat (default)
  -> residuals         : min = -0.5, mean = 0.0, max = 0.5
  -> model_info        : package sklearn

A new explainer has been created!
Preparation of a new explainer is initiated

  -> data              : 32559 rows 14 cols
  -> target variable   : Parameter 'y' was a pandas.DataFrame. Converted to

In [3]:
cat_feat = ['WorkClass', 'Education', 'MaritalStatus',
                                  'Occupation', 'Relationship', 'Race',
                                  'Gender', 'NativeCountry']
cont_feat = ['Age', 'fnlwgt', 'EducationNum',
                                  'CapitalGain', 'CapitalLoss',
                                  'HoursPerWeek']

In [6]:
# import dalex as dx
clfs_pd_cat = []
clfs_pd_cont = []
for clf_id, clf_name in enumerate(clfs):
    clf_pd_cat = clfs_explainers[clf_id].model_profile( variables = cat_feat,
                                                        variable_type='categorical')
    clfs_pd_cat.append(clf_pd_cat)

for clf_id, clf_name in enumerate(clfs):
    clf_pd_cont = clfs_explainers[clf_id].model_profile( variables = cont_feat)
    clfs_pd_cont.append(clf_pd_cont)

Calculating ceteris paribus: 100%|██████████| 6/6 [00:06<00:00,  1.09s/it]


In [7]:
# import dalex as dx
clfs_mp = []
for clf_id, clf_name in enumerate(clfs):
    clf_mp = clfs_explainers[clf_id].model_parts()
    clfs_mp.append(clf_mp)

In [19]:
import numpy as np 
adult_mp_results = np.zeros(shape=len(clfs))

for i in range(len(clfs)):
    adult_mp_results[i] = np.abs(np.subtract(clfs_mp[0].result.dropout_loss[1:-1], clfs_mp[i].result.dropout_loss[1:-1])).sum()

In [23]:
import pandas as pd
pd.DataFrame(adult_mp_results).to_csv(f"./results/mp_adult.csv")

In [33]:
adult_pd_cat_results = np.zeros(shape=len(clfs))
adult_pd_cont_results = np.zeros(shape=len(clfs))
for i in range(len(clfs)):
    adult_pd_cat_results[i] = np.abs(np.subtract(clfs_pd_cat[0].result._yhat_, clfs_pd_cat[i].result._yhat_)).sum()
    adult_pd_cont_results[i] = np.abs(np.subtract(clfs_pd_cont[0].result._yhat_, clfs_pd_cont[i].result._yhat_)).sum()

In [36]:
adult_pd_cat_results + adult_pd_cont_results

array([  0.        ,  40.18076222,  48.75556374,  36.78137434,
        58.33788   ,  29.70125   ,  41.517683  ,  58.96111682,
       237.70864414,  60.7272784 ,  78.32125162])

In [37]:
adult_pd_results = adult_pd_cat_results + adult_pd_cont_results

In [38]:
pd.DataFrame(adult_pd_results).to_csv(f"./results/pd_adult.csv")